In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import gower
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_predict, GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline, make_pipeline

In [38]:
df = pd.read_csv('hipertensao_normalizada.csv', sep=';')
#df.drop(columns=['X001'], inplace=True)
df.head()

,C008,C010,E001,E003,M016,N010,N011,N012,N013,N014,...,C009_branco,C009_preto,C009_amarelo,C009_pardo,C009_indigena,norte,nordeste,sudeste,sul,centro_oeste
0,0.714286,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.857143,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.428571,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.857143,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [39]:
# Iremos separar 20% da base para teste
# Com os outros 80%, aplicaremos um cross validation com fold = 5 (ficaria muito pequeno para fazer 10)
# Após isso, aplicaremos o oversample para cada dobra e então aplicar a rede neural para realizar o treinamento da classificação
y = df['Q002'].to_numpy()
df.drop(columns=['Q002'], inplace=True)
X = df.to_numpy()
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.25, random_state=0)

In [70]:
oversampler = SMOTE(random_state=42)
rede_neural = MLPClassifier()
# Nesta linha, estamos criando um pipeline para poder utilizar o cross-validation aplicando o oversampling SMOTE em cada dobra
imba_pipeline = make_pipeline(oversampler, rede_neural, verbose=True)

In [79]:
parametros = {
    'mlpclassifier__hidden_layer_sizes': [(128, 22)],
    'mlpclassifier__activation': ['relu'],
    'mlpclassifier__solver': ['sgd'],
    'mlpclassifier__learning_rate': ['adaptive'],
    'mlpclassifier__learning_rate_init': [0.001],
    'mlpclassifier__max_iter': [20000],
    #'mlpclassifier__verbose': [True],
    'mlpclassifier__random_state': [1]
}

In [80]:
grid_imba = GridSearchCV(imba_pipeline, param_grid=parametros, cv=5, scoring='recall', return_train_score=True)
grid_imba.fit(X_treino, y_treino);

[Pipeline] ............. (step 1 of 2) Processing smote, total=   0.0s
[Pipeline] ..... (step 2 of 2) Processing mlpclassifier, total=  42.8s
[Pipeline] ............. (step 1 of 2) Processing smote, total=   0.0s
[Pipeline] ..... (step 2 of 2) Processing mlpclassifier, total=  35.2s
[Pipeline] ............. (step 1 of 2) Processing smote, total=   0.0s
[Pipeline] ..... (step 2 of 2) Processing mlpclassifier, total=  35.3s
[Pipeline] ............. (step 1 of 2) Processing smote, total=   0.0s
[Pipeline] ..... (step 2 of 2) Processing mlpclassifier, total=  42.0s
[Pipeline] ............. (step 1 of 2) Processing smote, total=   0.0s
[Pipeline] ..... (step 2 of 2) Processing mlpclassifier, total=  38.2s
[Pipeline] ............. (step 1 of 2) Processing smote, total=   0.0s
[Pipeline] ..... (step 2 of 2) Processing mlpclassifier, total=  46.2s


In [81]:
grid_imba.best_params_

{'mlpclassifier__activation': 'relu',
 'mlpclassifier__hidden_layer_sizes': (128, 22),
 'mlpclassifier__learning_rate': 'adaptive',
 'mlpclassifier__learning_rate_init': 0.001,
 'mlpclassifier__max_iter': 20000,
 'mlpclassifier__random_state': 1,
 'mlpclassifier__solver': 'sgd'}

In [82]:
grid_imba.best_score_

0.12252100840336133

In [83]:
y_teste_predict = grid_imba.predict(X_teste)
print(classification_report(y_teste, y_teste_predict, target_names=['sem hipertensão', 'com hipertensão']))

                 precision    recall  f1-score   support

sem hipertensão       0.92      0.92      0.92       554
com hipertensão       0.12      0.12      0.12        49

       accuracy                           0.86       603
      macro avg       0.52      0.52      0.52       603
   weighted avg       0.86      0.86      0.86       603



In [84]:
confusion_matrix(y_teste, y_teste_predict)

array([[511,  43],
       [ 43,   6]], dtype=int64)

In [78]:
rede_neural.loss_curve_

AttributeError: 'MLPClassifier' object has no attribute 'loss_curve_'